In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import copy
from torchmetrics.functional import r2_score as torch_r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the data

In [103]:
df = pd.read_csv("/kaggle/input/us-college-completion-rate-analysis/train.csv")

# Removing outliers

Outlier Management

Outlier Removal was first attempted on the training dataset. However, after removal of outliers, only ~70% of the training dataset. This great reduction in the training dataset also reduces the model's prediction ability with less data to work with.

To maintain the training dataset size, capping outliers at the upper limit(75% Quartile + (1.5*InterQuartileRange)) and lower limit (25% Quartile - 1.5*(1.5*InterQuartileRange)) was selected.

In [80]:
def capping_outliers(X,num): #Expecting a labelled columns dataframe
    if num==1:
        X_cols = X.columns[2:]
        result = pd.DataFrame()
        result = pd.concat([result,pd.DataFrame(X['Completion_rate'])],axis=1)
    else:
        X_cols = X.columns[1:]
        result = pd.DataFrame()
    
    for i in X_cols:
        test = pd.DataFrame(X[i])
        
        Lower_Quartile = X[i].quantile(0.25)
        Upper_Quartile = X[i].quantile(0.75)
        IQR = Upper_Quartile- Lower_Quartile
        Upper_Limit = Upper_Quartile + (1.5*IQR)
        Lower_Limit = Lower_Quartile - (1.5*IQR)
        
        upper_array=np.array(X[i]>=Upper_Limit)
        lower_array=np.array(X[i]<=Lower_Limit)
        
        test.loc[upper_array] =Upper_Limit
        test.loc[lower_array] = Lower_Limit
        
        result = pd.concat([result,test],axis=1)
        
    return result

In [104]:
capped_outliers_data = capping_outliers(df,1)
capped_outliers_data

,Completion_rate,Tuition_in_state,Tuition_out_state,Faculty_salary,Pell_grant_rate,SAT_average,ACT_50thPercentile,pct_White,pct_Black,pct_Hispanic,pct_Asian,Parents_middlesch,Parents_highsch,Parents_college
0,0.6552,33560.0,33560.0,4285.0,0.2531,1029.0,21,0.3771,0.0612,0.0590,0.120137,0.011561,0.226397,0.762042
1,0.5246,7383.0,18543.0,9148.0,0.4752,963.0,21,0.2866,0.0331,0.2639,0.101100,0.063970,0.375165,0.517424
2,0.4030,23144.0,23144.0,7232.0,0.4668,984.0,21,0.3085,0.1426,0.2309,0.014900,0.063970,0.361314,0.547445
3,0.5472,6610.0,17770.0,9284.0,0.6369,877.0,18,0.1272,0.0572,0.2639,0.055700,0.063970,0.433621,0.413556
4,0.8166,47004.0,47004.0,11984.0,0.1337,1323.0,30,0.4251,0.0786,0.2268,0.054900,0.014868,0.186091,0.799041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,0.5311,24090.0,24090.0,6705.0,0.3402,949.0,20,0.6830,0.0544,0.0385,0.022500,0.019266,0.439450,0.541284
2360,0.5371,36660.0,36660.0,6340.0,0.4075,1033.0,23,0.6215,0.1582,0.0883,0.025700,0.028593,0.404816,0.566591
2361,0.3677,30331.0,30331.0,6689.0,0.5791,1009.0,22,0.5842,0.2277,0.0569,0.022100,0.029851,0.370421,0.599729
2362,0.5541,5055.0,16215.0,9312.0,0.2151,1102.0,24,0.7347,0.0113,0.0718,0.011700,0.017512,0.259759,0.722729


In [105]:
feature_names = capped_outliers_data.columns[1:]
feature_names

Index(['Tuition_in_state', 'Tuition_out_state', 'Faculty_salary',
       'Pell_grant_rate', 'SAT_average', 'ACT_50thPercentile', 'pct_White',
       'pct_Black', 'pct_Hispanic', 'pct_Asian', 'Parents_middlesch',
       'Parents_highsch', 'Parents_college'],
      dtype='object')

In [106]:
features = ['Tuition_in_state','Tuition_out_state','Faculty_salary','Pell_grant_rate','SAT_average','ACT_50thPercentile','pct_White','pct_Black','pct_Hispanic','pct_Asian','Parents_middlesch','Parents_highsch','Parents_college'] 
X = capped_outliers_data[features]
y = capped_outliers_data["Completion_rate"]

#Comment out when sending to Kaggle
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Preprocessing

In [107]:
#Comment out when training
scaler = StandardScaler()
scaler.fit(X)
X_train = scaler.transform(X)

In [ ]:
#Comment out when sending to Kaggle
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [96]:
#Real X_Test
real_X_test = pd.read_csv("/kaggle/input/us-college-completion-rate-analysis/x_test.csv")
real_X_test = real_X_test.drop(columns=['Unnamed: 0'])
real_X_test = scaler.transform(real_X_test)

## XGBOOST Hyperparameter tuning

In [44]:
# xgb_param_grid = { 
#     'max_depth' : [6,20,50,80],
#     'learning_rate' :[0.01,0.1,1],
#     'colsample_bytree':[0.5,0.75,1],
#     'subsample': [0.5,0.75,1]
    
    
# }

# xgb_tuned = xgb.XGBRegressor()
# CV_xgb_REG = GridSearchCV(estimator=xgb_tuned, param_grid=xgb_param_grid, scoring ='neg_root_mean_squared_error')
# CV_xgb_REG.fit(X_train, y_train)
# CV_xgb_REG.best_params_

#Output:
# {'colsample_bytree': 1,
#  'learning_rate': 0.1,
#  'max_depth': 50,
#  'subsample': 0.75}

{'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 50,
 'subsample': 0.75}

In [45]:
# xgb_param_grid = { 
#     'max_depth' : [30,40,50,60],
#     'learning_rate' :[0.05,0.1,0.2],
#     'colsample_bytree':[0.8,0.9,1],
#     'subsample': [0.65,0.75,0.85]
    
    
# }

# xgb_tuned = xgb.XGBRegressor()
# CV_xgb_REG = GridSearchCV(estimator=xgb_tuned, param_grid=xgb_param_grid, scoring ='neg_root_mean_squared_error')
# CV_xgb_REG.fit(X_train, y_train)
# CV_xgb_REG.best_params_

#Output:
# {'colsample_bytree': 0.9,
#  'learning_rate': 0.05,
#  'max_depth': 30,
#  'subsample': 0.65}

{'colsample_bytree': 0.9,
 'learning_rate': 0.05,
 'max_depth': 30,
 'subsample': 0.65}

In [47]:
# xgb_param_grid = { 
#     'max_depth' : [15,20,25,30,35],
#     'learning_rate' :[0.03,0.04,0.05,0.06,0.07,0.08],
#     'colsample_bytree':[0.9],
#     'subsample': [0.65]
    
    
# }

# xgb_tuned = xgb.XGBRegressor()
# CV_xgb_REG = GridSearchCV(estimator=xgb_tuned, param_grid=xgb_param_grid, scoring ='neg_root_mean_squared_error')
# CV_xgb_REG.fit(X_train, y_train)
# CV_xgb_REG.best_params_

#Output:
# {'colsample_bytree': 0.9,
#  'learning_rate': 0.06,
#  'max_depth': 25,
#  'subsample': 0.65}

{'colsample_bytree': 0.9,
 'learning_rate': 0.06,
 'max_depth': 25,
 'subsample': 0.65}

## Optimizing n_estimators parameter

n_estimators = 100

In [58]:
# XGB_model = xgb.XGBRegressor(n_estimators=100,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
# XGB_model.fit(X_train, y_train)
# xgb_r_sq = XGB_model.score(X_train, y_train)
# print(xgb_r_sq)

#0.9988690551198027

0.9988690551198027


In [59]:
# #Using model to obtain predictions for the Train Test split
# y_pred_TEST_XGB_model = XGB_model.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST_XGB_model, squared=False)
# RMSE

#0.06491004234150678

0.06491004234150678

n_estimators =200

In [60]:
# XGB_model = xgb.XGBRegressor(n_estimators=200,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
# XGB_model.fit(X_train, y_train)
# xgb_r_sq = XGB_model.score(X_train, y_train)
# print(xgb_r_sq)

#0.999981317382156

0.999981317382156


In [61]:
# #Using model to obtain predictions for the Train Test split
# y_pred_TEST_XGB_model = XGB_model.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST_XGB_model, squared=False)
# RMSE

#0.06423899265251357

0.06423899265251357

n_estimators = 300

In [62]:
# XGB_model = xgb.XGBRegressor(n_estimators=300,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
# XGB_model.fit(X_train, y_train)
# xgb_r_sq = XGB_model.score(X_train, y_train)
# print(xgb_r_sq)

#0.9999914656642858

0.9999914656642858


In [63]:
# #Using model to obtain predictions for the Train Test split
# y_pred_TEST_XGB_model = XGB_model.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST_XGB_model, squared=False)
# RMSE

#0.0642247993077135

0.0642247993077135

n_estimators = 400

In [64]:
# XGB_model = xgb.XGBRegressor(n_estimators=400,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
# XGB_model.fit(X_train, y_train)
# xgb_r_sq = XGB_model.score(X_train, y_train)
# print(xgb_r_sq)

#0.9999922930303264

0.9999922930303264


In [65]:
# #Using model to obtain predictions for the Train Test split
# y_pred_TEST_XGB_model = XGB_model.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST_XGB_model, squared=False)
# RMSE

#0.06422117886906899

0.06422117886906899

n_estimators = 500

In [66]:
# XGB_model = xgb.XGBRegressor(n_estimators=500,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
# XGB_model.fit(X_train, y_train)
# xgb_r_sq = XGB_model.score(X_train, y_train)
# print(xgb_r_sq)

#0.9999925382189968

0.9999925382189968


In [67]:
# #Using model to obtain predictions for the Train Test split
# y_pred_TEST_XGB_model = XGB_model.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST_XGB_model, squared=False)
# RMSE

#0.0642215882290082

0.0642215882290082

Even though the training R_squared (0.9999925382189968)score of the model when n_estimators = 500 is greater than that of the training R_squared score (0.9999922930303264) of the model when n_estimators = 400 , the test RMSE (0.0642215882290082) of the model when n_estimators = 500 is greater than the test RMSE (0.06422117886906899) of the model when n_estimators = 400, this shows overfitting when n_estimators = 500 and the final model for the Capping Outliers XGB Regressor model has the following parameters:

n_estimators=400,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9

### Creating Predictions for Kaggle dataset

In [108]:
real_X_test = pd.read_csv("/kaggle/input/us-college-completion-rate-analysis/x_test.csv")
# real_X_test = capping_outliers(real_X_test,2)
real_X_test = real_X_test.drop(columns=['Unnamed: 0'])
real_X_test = scaler.transform(real_X_test)

In [85]:
XGB_model = xgb.XGBRegressor(n_estimators=400,subsample = 0.65, max_depth = 25, learning_rate = 0.06, colsample_bytree = 0.9)
XGB_model.fit(X_train, y.to_numpy())
true_y_pred_TEST_XGB_tuned = XGB_model.predict(real_X_test)
print(true_y_pred_TEST_XGB_tuned)

[0.42521465 0.46355394 0.4886781  ... 0.5772263  0.5732011  0.6491689 ]


### Creating our submission

In [86]:
submission = pd.DataFrame.from_dict({'Completion_rate': true_y_pred_TEST_XGB_tuned})
submission

,Completion_rate
0,0.425215
1,0.463554
2,0.488678
3,0.426610
4,0.500699
...,...
1160,0.501211
1161,0.484570
1162,0.577226
1163,0.573201


In [87]:
submission.to_csv('submission.csv', index = True, index_label = "id") 

### To submit your submission

On the panel on your right, select the drop down "Submit to competition", and submit this notebook. Ensure that your submission is named ```submission.csv```!